In [1]:
from Utilities.DRNN import DRNN
import torch.nn as nn
import torch

class TestBDRNN(nn.Module):
    def __init__(self, input_size, sequence_length, hidden_size, dilations):
        super(TestBDRNN, self).__init__()

        self._input_size = input_size
        self._sequence_length = sequence_length
        self._hidden_size = hidden_size
        self._dilations = dilations
        self._number_of_units = hidden_size[-1]

        self._rnn = DRNN(input_size, hidden_size,
                           dilations=dilations, batch_first=True)
        self._linear = nn.Linear(self._number_of_units, input_size)

    def forward(self, inputs):
        out, hidden = self._rnn(inputs)
        out = out[-1] # last layer
        outputs = []
        for index in range(out.shape[1]):
            current_out = out[:, index, :]
            current_out = self._linear(current_out)
            current_out = current_out.repeat(1, 1, 1).transpose(0, 1)

            outputs.append(current_out)
        
        output = torch.cat(outputs, dim=1)
        return output

In [2]:

from torch.utils.data import DataLoader
from Utilities.DTCR import DTCRModel, DTCRConfig
from Utilities.UCRParser import read_dataset
from Utilities.DRNN import BidirectionalDRNN

train_ds, test_ds = read_dataset("Two_Patterns")
bs = 100
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=bs, shuffle=True)

model = TestBDRNN(train_ds[0][0].shape[1], train_ds[0][0].shape[0], [100, 50, 50], [1, 4, 16])
mse = torch.nn.MSELoss()
opt = torch.optim.Adam(list(model.parameters()))#, eps=5e-3)
print_interval = 5
for epoch in range(2):
    running_loss = 0.0
    for index, (sample_data, sample_label) in enumerate(train_dl):
        opt.zero_grad()

        outputs = model(sample_data)
        reconstruction_loss = mse(outputs, sample_data)
        reconstruction_loss.backward()
        opt.step()

        running_loss += reconstruction_loss.item()
        if index % print_interval == print_interval - 1:
            print('[%d, %5d] loss: %.3f' %
                (epoch + 1, index + 1, running_loss / print_interval))
            print("Real values:")
            print(",".join(["{:.2f}".format(float(item)) for item in sample_data[0]][:15]))
            print("Predicted:")
            print(",".join(["{:.2f}".format(float(item)) for item in outputs[0]][:15]))
            
            running_loss = 0.0


Loading the Two_Patterns dataset...
The dataset Two_Patterns was loaded.
[1,     5] loss: 0.978
Real values:
-0.10,0.12,-0.23,0.06,-0.35,-0.22,0.36,-0.22,0.09,0.17,-0.49,-0.14,0.10,0.15,-0.10
Predicted:
0.04,0.03,0.03,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
[1,    10] loss: 0.924
Real values:
-0.46,0.28,-0.15,0.05,0.42,0.17,-0.16,-0.12,0.22,-0.48,-0.55,0.51,-0.05,0.09,0.33
Predicted:
0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.02,0.00,-0.01,0.00,0.01,0.00,0.01
[2,     5] loss: 0.825
Real values:
0.65,0.16,0.31,0.27,-0.03,-0.69,-0.07,-0.26,1.49,1.49,1.49,1.49,1.49,1.49,1.49
Predicted:
0.06,0.06,0.07,0.07,0.06,0.01,0.01,0.00,0.09,0.14,0.19,0.22,0.27,0.31,0.33
[2,    10] loss: 0.657
Real values:
0.11,-0.08,0.89,0.36,-0.31,0.33,-0.26,-0.20,0.38,-0.55,0.18,0.15,-0.19,-0.28,-0.40
Predicted:
0.03,0.02,0.10,0.13,0.07,0.08,0.06,0.03,0.04,-0.02,-0.01,-0.00,-0.02,-0.06,-0.10


In [3]:
print_interval = 5
for epoch in range(10):
    running_loss = 0.0
    for index, (sample_data, sample_label) in enumerate(train_dl):
        opt.zero_grad()

        outputs = model(sample_data)
        reconstruction_loss = mse(outputs, sample_data)
        reconstruction_loss.backward()
        opt.step()

        running_loss += reconstruction_loss.item()
        if index % print_interval == print_interval - 1:
            print('[%d, %5d] loss: %.3f' %
                (epoch + 1, index + 1, running_loss / print_interval))
            print("Real values:")
            print(",".join(["{:.2f}".format(float(item)) for item in sample_data[0]][:15]))
            print("Predicted:")
            print(",".join(["{:.2f}".format(float(item)) for item in outputs[0]][:15]))
            
            running_loss = 0.0


[1,     5] loss: 0.428
Real values:
-0.40,-0.33,0.37,0.07,-0.18,-0.01,-0.57,-0.15,-0.24,-0.48,-0.39,-0.06,-0.49,0.41,0.37
Predicted:
-0.05,-0.11,-0.03,0.00,-0.03,-0.04,-0.12,-0.13,-0.16,-0.22,-0.27,-0.23,-0.26,-0.12,0.01
[1,    10] loss: 0.226
Real values:
-0.24,0.38,-0.63,-0.47,-0.61,-0.27,0.13,0.69,-0.02,0.28,-0.04,0.20,-0.16,-0.09,0.19
Predicted:
-0.05,0.06,-0.11,-0.24,-0.41,-0.40,-0.25,0.14,0.26,0.38,0.35,0.36,0.19,0.02,-0.02
[2,     5] loss: 0.153
Real values:
0.00,0.30,-0.00,0.40,-0.21,0.14,0.42,-0.25,-0.69,0.05,-0.19,0.07,-0.18,0.47,-0.25
Predicted:
0.01,0.13,0.13,0.29,0.15,0.14,0.28,0.09,-0.39,-0.39,-0.37,-0.20,-0.13,0.27,0.17
[2,    10] loss: 0.107
Real values:
0.17,-0.01,-0.43,-0.28,0.27,-0.14,-0.12,0.56,-0.30,0.10,0.26,-0.07,0.11,0.13,0.03
Predicted:
0.07,0.05,-0.18,-0.29,-0.01,0.02,-0.03,0.34,0.12,0.07,0.17,0.08,0.05,0.09,0.08
[3,     5] loss: 0.056
Real values:
0.19,-0.30,0.30,-0.36,-0.22,0.25,-0.14,0.33,0.18,0.44,-0.39,0.58,-0.70,0.08,0.04
Predicted:
0.08,-0.11,0.09,-0.13